# How prepare data from stweet to find geolocation of users?

In [1]:
import pandas as pd

In [2]:
df_users = pd.read_json('data/users_data_2021_12_09.json')

## Nas interesuje kolumna 'location' oraz informacje, które możemy z niej wyciągnąć
Z niej spróbuj coś wyciągnąć  `geopy.geocoders` na tyle ile ci się uda (https://geopy.readthedocs.io/en/release-0.96.3/) 

tutorial: (https://ecyy.medium.com/geocoding-and-reversegeocoding-by-geopy-in-colab-b8c43f7261f9)

In [41]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent = "stance_detection_for_vaccin")

In [44]:
location = geolocator.geocode('Around the world')
print(location)
print((location.latitude, location.longitude))

Around the World, Whistler Olympic Park, Area D (Elaho/Garibaldi), Squamish-Lillooet Regional District, British Columbia, Canada
(50.1324227, -123.1174466)


A tu masz kolumne z danymi

In [22]:
df_users['location']

0                 Polska
1                       
2                Arcelin
3           Łódź, Poland
4                       
              ...       
9753           Stockholm
9754    Around the world
9755              CANADA
9756      Washington, DC
9757     London, England
Name: location, Length: 9758, dtype: object

## Druga opcja - dane lokalizacyjne powiązane z konkretnym tweetem - niestety nie wypaliły
Jak zauważysz ponizej to mamy tylko 51 informacji na 36k tweetów. Czyli 'nic'

In [8]:
df_tweet = pd.read_json('data/tweets_data_2021_12_09.json')

In [9]:
df_tweet.columns

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'main_hashtags',
       'user_mention', 'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user_id', 'user_id_str', 'geo',
       'coordinates', 'place', 'contributors', 'is_quote_status',
       'retweet_count', 'favorite_count', 'reply_count', 'quote_count',
       'conversation_id', 'favorited', 'retweeted', 'possibly_sensitive',
       'lang'],
      dtype='object')

In [25]:
len(df_tweet['coordinates'].dropna())

51

In [21]:
len(df_tweet['geo'].dropna())

51

## Do pracy z danymi przestrzennymi mamy parę bibliotek:
1. Geopandas (główna i fajnie ją ogarnąć, bo na niej Szymański głównie bazuje)
- Do działania wszystkiego potrzebujesz 2 biblioteki: GDAL, Fiona. Obie niestety trzeba instalować ręcznie (na windowsie na pewno, na linuxie czasami idzie automatycznie)
- jak będziesz miał z tym problemy to przerzuć sobie pliki na colaba, tam instalacja idzie automatycznie
- wizualizacja z wykorzytaniem geopandas i matplotliba: https://towardsdatascience.com/how-to-convert-a-shapefile-to-a-dataframe-in-python-a6ca9a893504



In [ ]:
############## Poradnik dla opornych
## Do instalacji GDAL - https://www.lfd.uci.edu/~gohlke/pythonlibs/#gdal
# !pip install GDAL-3.4.1-pp38-pypy38_pp73-win_amd64.whl
## Do instalacji bibliotekę Fiona (https://www.lfd.uci.edu/~gohlke/pythonlibs/#fiona) (tutorial: https://stackoverflow.com/questions/50876702/cant-install-fiona-on-windows)
# !pip install <path/to/Fiona.whl>
# !pip install geopandas

2. Na potrzeby wizualizacji cość ładną biblioteką jest folium (pip install folium) i poniżej powinienes mieć jakiś przykład działania

tutorial: (https://medium.com/datasciencearth/map-visualization-with-folium-d1403771717)

Co do wizualizacji fajnie by było ogarnąć, czy można zrobić jakiś ładny podział na województwa (jakiś geojson czy coś)

In [63]:
import folium 
from folium import plugins
geolocator = Nominatim(timeout=10, user_agent = "stance_detection_for_vaccin")
location = geolocator.geocode('Łódź')
map = folium.Map(location=[location.latitude,location.longitude],
                    width="%100",height="%100",
                    zoom_start=8)
location = geolocator.geocode('Smolec')
folium.Marker(location=[location.latitude,location.longitude], popup ='Smolec').add_to(map) #to są znaczniki, w pętli można je dodawać
location = geolocator.geocode('Radom')
folium.Marker(location=[location.latitude,location.longitude], popup ='Radom').add_to(map) #to są znaczniki, w pętli można je dodawać
minimap = plugins.MiniMap()
map.add_child(minimap)
map


3. (najtrudniejsza rzecz) Musimy postawić model do naszych danych przestrzennych  - do omówienia